In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recommended-system-for-amazon-icl-2021/train.csv
/kaggle/input/recommended-system-for-amazon-icl-2021/test.csv


In [31]:
import pandas as pd
import numpy as np
dtype = [("userId", np.string_), ("movieId", np.string_), ("rating", np.int32)]

In [32]:
train_dataset = pd.read_csv('/kaggle/input/recommended-system-for-amazon-icl-2021/train.csv',dtype=dict(dtype))
train_dataset.head(5)

,userId,productId,Rating
0,AOPE42H34R0EC,B00000DM9W,5.0
1,A1GI09JC6L0NF7,B00004SABJ,4.0
2,AZLZII4AFX56R,B00000J579,3.0
3,A34AHNT6GD9FWW,9888002198,5.0
4,A2PXRAO5C1XTLW,0972683275,5.0


In [33]:

# 评分预测    1-5
class LFM(object):

    def __init__(self, alpha, reg_p, reg_q, number_LatentFactors=10, number_epochs=10, columns=["uid", "iid", "rating"]):
        self.alpha = alpha # 学习率
        self.reg_p = reg_p    # P矩阵正则
        self.reg_q = reg_q    # Q矩阵正则
        self.number_LatentFactors = number_LatentFactors  # 隐式类别数量
        self.number_epochs = number_epochs    # 最大迭代次数
        self.columns = columns

    def fit(self, dataset):
        '''
        fit dataset
        :param dataset: uid, iid, rating
        :return:
        '''

        self.dataset = pd.DataFrame(dataset)

        self.users_ratings = dataset.groupby(self.columns[0]).agg([list])[[self.columns[1], self.columns[2]]]
        self.items_ratings = dataset.groupby(self.columns[1]).agg([list])[[self.columns[0], self.columns[2]]]

        self.globalMean = self.dataset[self.columns[2]].mean()

        self.P, self.Q = self.sgd()

    def _init_matrix(self):
        '''
        初始化P和Q矩阵，同时为设置0，1之间的随机值作为初始值
        :return:
        '''
        # User-LF
        P = dict(zip(
            self.users_ratings.index,
            np.random.rand(len(self.users_ratings), self.number_LatentFactors).astype(np.float32)
        ))
        # Item-LF
        Q = dict(zip(
            self.items_ratings.index,
            np.random.rand(len(self.items_ratings), self.number_LatentFactors).astype(np.float32)
        ))
        return P, Q

    def sgd(self):
        '''
        使用随机梯度下降，优化结果
        :return:
        '''
        P, Q = self._init_matrix()

        for i in range(self.number_epochs):
            print("iter%d"%i)
            error_list = []
            for uid, iid, r_ui in self.dataset.itertuples(index=False):
                # User-LF P
                ## Item-LF Q
                v_pu = P[uid] #用户向量
                v_qi = Q[iid] #物品向量
                err = np.float32(r_ui - np.dot(v_pu, v_qi))

                v_pu += self.alpha * (err * v_qi - self.reg_p * v_pu)
                v_qi += self.alpha * (err * v_pu - self.reg_q * v_qi)
                
                P[uid] = v_pu 
                Q[iid] = v_qi

                # for k in range(self.number_of_LatentFactors):
                #     v_pu[k] += self.alpha*(err*v_qi[k] - self.reg_p*v_pu[k])
                #     v_qi[k] += self.alpha*(err*v_pu[k] - self.reg_q*v_qi[k])

                error_list.append(err ** 2)
            print(np.sqrt(np.mean(error_list)))
        return P, Q

    def predict(self, uid, iid):
        # 如果uid或iid不在，我们使用全剧平均分作为预测结果返回
        if uid not in self.users_ratings.index or iid not in self.items_ratings.index:
            return self.globalMean

        p_u = self.P[uid]
        q_i = self.Q[iid]

        return np.dot(p_u, q_i)

    def test(self,testset):
        '''预测测试集数据'''
        df=pd.DataFrame({"Keys":[],"Rating":[]})
        i=0;
        for uid, iid, _ in testset.itertuples(index=False):
            pred_rating = self.predict(uid, iid)
            pred_rating = round(pred_rating)
            pred_rating = round(pred_rating,2)
            new_row=pd.Series({"Keys":(uid,iid),"Rating":pred_rating})
            df = pd.concat([df, new_row.to_frame().T])
            i =i +1
        df.to_csv('/kaggle/working/submission.csv',index=False)
                
           

SyntaxError: invalid character '，' (U+FF0C) (1641194699.py, line 97)

In [ ]:
test_dataset = pd.read_csv('/kaggle/input/recommended-system-for-amazon-icl-2021/test.csv',dtype=dict(dtype))
lfm = LFM(0.02, 0.01, 0.01, 10, 50, ["userId","productId","Rating"])
lfm.fit(train_dataset)
lfm.test(test_dataset)